<a href="https://colab.research.google.com/github/urszkam/AoC_2022/blob/main/Copy_of_spark_ml_student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zadanie: Budowa modelu regresji liniowej liniowej



- **Identyfikacja:**
  - `ID`: Unikalny identyfikator wypadku.
  - `Source`: Źródło danych (np. raporty policyjne, systemy monitoringu).

- **Czas:**
  - `Start_Time`, `End_Time`: Czas rozpoczęcia i zakończenia wypadku.

- **Lokalizacja:**
  - `Start_Lat`, `Start_Lng`, `End_Lat`, `End_Lng`: Współrzędne geograficzne miejsca wypadku.
  - `Distance(mi)`: Dystans, jaki obejmuje wypadek.
  - `City`, `State`, `Zipcode`, `Country`: Informacje o lokalizacji administracyjnej.
  - `Street`: Ulica, na której doszło do wypadku.

- **Pogoda:**
  - `Weather_Condition`: Warunki pogodowe (np. "Rain", "Clear").
  - `Temperature(F)`, `Humidity(%)`, `Visibility(mi)`: Kluczowe dane atmosferyczne.
  - `Wind_Speed(mph)`, `Wind_Direction`: Prędkość i kierunek wiatru.
  - `Precipitation(in)`: Opady deszczu lub śniegu.

- **Infrastruktura:**
  - `Amenity`, `Bump`, `Crossing`, `Traffic_Signal`: Informacje o obiektach i elementach drogowych w pobliżu wypadku (np. ronda, progi zwalniające, skrzyżowania).
  - `Junction`, `No_Exit`, `Railway`: Lokalizacje związane z infrastrukturą drogową.

- **Oświetlenie:**
  - `Sunrise_Sunset`: Pora dnia w chwili wypadku (np. "Day" lub "Night").
  - `Civil_Twilight`, `Nautical_Twilight`, `Astronomical_Twilight`: Różne fazy zmierzchu i świtu.

- **Opis i szczegóły:**
  - `Description`: Tekstowy opis wypadku.
  - `Severity`: Skala powagi wypadku (np. od 1 do 5).
  - `Airport_Code`: Kod lotniska najbliższego miejsca wypadku.

---


Twoim zadaniem jest:

1. Wczytanie danych o wypadkach drogowych w USA z pliku CSV do Apache Spark.
2. Analiza struktury danych i wybór odpowiednich cech.
3. Budowa modelu regresji liniowej w celu przewidywania powagi wypadków (Severity).
4. Ocena jakości modelu.

#### Opis zadania:

Wczytaj dane:

1. Użyj dostarczonego kodu, aby wczytać plik CSV o nazwie ```US_Accidents_March23.csv``` do Apache Spark jako DataFrame.
2. Wyświetl schemat danych (printSchema) i pierwsze wiersze tabeli (show()), aby zapoznać się z zawartością.
3. Przygotowanie danych:

Wybierz następujące cechy (features):
* Temperature(F)
* Humidity(%)
* Visibility(mi)
* Wind_Speed(mph)
* Precipitation(in)
* Distance(mi)
Usuń wiersze z brakującymi wartościami w wybranych kolumnach.

#### Budowa modelu:

1. Użyj VectorAssembler, aby połączyć wybrane cechy w jedną kolumnę features.
2. Stwórz model regresji liniowej (LinearRegression), gdzie:
3. Kolumna features będzie wejściem.
4. Kolumna Severity będzie celem (target).

####Podział danych:

1. Podziel dane na zbiór treningowy (80%) i testowy (20%).
2. Trening i predykcja:
3. Wytrenuj model na zbiorze treningowym.
4. Wykonaj predykcje na zbiorze testowym.

#### Ewaluacja modelu:

1. Oblicz i wyświetl metrykę RMSE (Root Mean Squared Error).
2. Wyświetl współczynniki regresji oraz wyraz wolny (intercept).
3. Pytania do odpowiedzi:
4. Jakie są współczynniki regresji dla wybranych cech?
5. Jakie jest RMSE dla modelu? Czy wynik jest satysfakcjonujący?
6. Jakie kroki można podjąć, aby poprawić jakość modelu?


In [4]:
import kagglehub

path = kagglehub.dataset_download("sobhanmoosavi/us-accidents")
print(path)

Using Colab cache for faster access to the 'us-accidents' dataset.
/kaggle/input/us-accidents


In [ ]:
!cp -R /root/.cache/kagglehub/datasets/sobhanmoosavi/us-accidents/versions/13/US_Accidents_March23.csv /content/sample_data/US_Accidents_March23.csv


In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
                    .master("local[*]") \
                    .config("spark.executor.memory", "8g") \
                    .config("spark.driver.memory", "4g") \
                    .appName("mlibs") \
                    .getOrCreate()

inDF = spark.read.format("csv") \
  .option("sep", ",") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load(path + "/US_Accidents_March23.csv")

inDF.printSchema()

inDF.createOrReplaceTempView("accidents")

root
 |-- ID: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Severity: integer (nullable = true)
 |-- Start_Time: timestamp (nullable = true)
 |-- End_Time: timestamp (nullable = true)
 |-- Start_Lat: double (nullable = true)
 |-- Start_Lng: double (nullable = true)
 |-- End_Lat: double (nullable = true)
 |-- End_Lng: double (nullable = true)
 |-- Distance(mi): double (nullable = true)
 |-- Description: string (nullable = true)
 |-- Street: string (nullable = true)
 |-- City: string (nullable = true)
 |-- County: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Timezone: string (nullable = true)
 |-- Airport_Code: string (nullable = true)
 |-- Weather_Timestamp: timestamp (nullable = true)
 |-- Temperature(F): double (nullable = true)
 |-- Wind_Chill(F): double (nullable = true)
 |-- Humidity(%): double (nullable = true)
 |-- Pressure(in): double (nullable = true)
 |-- V

+--------------------+--------+------------------+
|            features|Severity|        prediction|
+--------------------+--------+------------------+
|[5.0,84.0,2.5,6.9...|       2| 2.185137071886402|
|[5.0,84.0,5.0,6.9...|       2|2.1800807966217635|
|[6.1,55.0,7.0,11....|       2| 2.171018199493843|
|[7.0,53.0,10.0,15...|       2| 2.171207438083971|
|[7.0,80.0,5.0,5.8...|       3|2.1752611988035584|
|[8.1,46.0,10.0,11...|       2| 2.159909903403458|
|[9.0,67.0,9.0,18....|       2|2.2088813287510263|
|[9.0,77.0,2.5,8.1...|       3|2.1833681536096514|
|[9.0,84.0,6.0,6.9...|       2| 2.177109464412685|
|[9.0,84.0,6.0,6.9...|       2| 2.177109464412685|
+--------------------+--------+------------------+
only showing top 10 rows

Root Mean Squared Error (RMSE): 0.45
Współczynniki regresji: [-0.0002182711982496193,0.000504367297982355,-0.002044149337348125,0.0021505329361605386,0.06988612086376723,0.005409807873193927]
Wyraz wolny (intercept): 2.134133270930995


In [7]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression


In [12]:
input_cols = [
    "Temperature(F)",
    "Humidity(%)",
    "Visibility(mi)",
    "Wind_Speed(mph)",
    "Precipitation(in)",
    "Distance(mi)",
]
label = "Severity"

clean_inDF = inDF.na.drop(subset=[label, *input_cols])
df = clean_inDF.select([label, *input_cols])

df.show(2)

+--------+--------------+-----------+--------------+---------------+-----------------+------------+
|Severity|Temperature(F)|Humidity(%)|Visibility(mi)|Wind_Speed(mph)|Precipitation(in)|Distance(mi)|
+--------+--------------+-----------+--------------+---------------+-----------------+------------+
|       3|          37.9|       97.0|           7.0|            3.5|             0.03|        0.01|
|       3|          37.4|      100.0|           3.0|            4.6|             0.02|        0.01|
+--------+--------------+-----------+--------------+---------------+-----------------+------------+
only showing top 2 rows



In [16]:
assembler = VectorAssembler(inputCols=input_cols, outputCol="features")
lr = LinearRegression(featuresCol="features", labelCol=label)
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

In [17]:
pipeline = Pipeline(stages=[assembler, lr])
model = pipeline.fit(df)


In [18]:
predictions = model.transform(df)

In [19]:
predictions.select("features", label, "prediction").show()

+--------------------+--------+------------------+
|            features|Severity|        prediction|
+--------------------+--------+------------------+
|[37.9,97.0,7.0,3....|       3|2.1709560176596714|
|[37.4,100.0,3.0,4...|       3| 2.182876339769181|
|[37.4,100.0,3.0,4...|       3| 2.182876339769181|
|[37.4,100.0,3.0,4...|       2| 2.182876339769181|
|[33.8,100.0,2.0,4...|       2| 2.184666462953303|
|[35.1,89.0,6.0,8....|       2| 2.180014716893894|
|[37.0,96.0,8.0,5....|       3|2.1724807697553956|
|[37.9,100.0,5.0,4...|       2| 2.186724965931995|
|[36.0,86.0,7.0,9....|       2| 2.177354724010093|
|[39.9,70.0,10.0,1...|       2| 2.166800138628533|
|[37.0,89.0,10.0,6...|       2| 2.167133889906331|
|[34.2,97.0,10.0,3...|       2| 2.165077596457728|
|[33.1,92.0,7.0,8....|       2|2.1788551503058566|
|[33.1,92.0,9.0,9....|       2|2.1850641984069146|
|[22.8,89.0,4.0,11...|       2| 2.193968815327794|
|[26.6,86.0,1.5,8....|       3|2.1876426119306043|
|[21.0,85.0,0.8,9....|       2|

In [22]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator_rmse = RegressionEvaluator(labelCol=label, predictionCol="prediction", metricName="rmse")
rmse = evaluator_rmse.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE) on test data = {rmse}")

evaluator_r2 = RegressionEvaluator(labelCol=label, predictionCol="prediction", metricName="r2")
r2 = evaluator_r2.evaluate(predictions)
print(f"R^2 on test data = {r2}")

Root Mean Squared Error (RMSE) on test data = 0.44982162353196187
R^2 on test data = 0.003757560377714575


In [26]:
lr_model = model.stages[-1]
coefs = list(lr_model.coefficients)
intercept = lr_model.intercept

In [37]:
for name, coef in zip(input_cols, coefs):
    print(f"{name}: {coef:+.6f}")
print(f"Intercept: {intercept:+.6f}")

Temperature(F): -0.000198
Humidity(%): +0.000641
Visibility(mi): -0.001922
Wind_Speed(mph): +0.002716
Precipitation(in): +0.077877
Distance(mi): +0.006542
Intercept: +2.117780


# Zadanie 2

#### Zadanie 1: Wczytanie danych i ich analiza
1. Wczytaj dane o wypadkach z pliku CSV do Apache Spark: Plik: ```US_Accidents_March23.csv```.
Użyj dostarczonego kodu do inicjalizacji SparkSession i wczytania pliku.

#### Zadanie 2: Przygotowanie danych
1. Wybierz następujące cechy (features) do modelu:
Temperature(F), Humidity(%), Visibility(mi), Wind_Speed(mph), Precipitation(in), Distance(mi).
2. Usuń wiersze z brakującymi wartościami w wybranych kolumnach.
3. Jeśli Severity jest zmienną kategoryczną, zakoduj ją jako wartości liczbowe za pomocą StringIndexer.

#### Zadanie 3: Budowa modelu drzewa decyzyjnego
1. Przygotuj dane wejściowe:
2. Połącz wybrane cechy w jedną kolumnę features za pomocą VectorAssembler.
3. Stwórz model drzewa decyzyjnego (DecisionTreeClassifier) o maksymalnej głębokości 5.
4. Podziel dane na zbiór treningowy (80%) i testowy (20%).
Wytrenuj model na zbiorze treningowym.

#### Zadanie 4: Ewaluacja modelu
1. Wykonaj predykcje na zbiorze testowym.
2. Wyświetl wyniki predykcji w formacie:
features, label (rzeczywista klasa), prediction (przewidywana klasa).
3. Oblicz dokładność modelu (accuracy) za pomocą MulticlassClassificationEvaluator.

#### Zadanie 5: Analiza wyników
1. Wyświetl strukturę drzewa decyzyjnego (toDebugString).
Odpowiedz na pytania:
* Jakie cechy są najbardziej istotne w podziale danych w drzewie?
* Czy dokładność modelu jest satysfakcjonująca? Jeśli nie, jakie mogą być przyczyny?
#### Zadanie 6: Optymalizacja modelu
1. Zastosuj optymalizację hiperparametrów:
2. Użyj CrossValidator i ParamGridBuilder, aby przetestować różne wartości maxDepth (np. [3, 5, 7]) i maxBins (np. [16, 32, 64]).
3. Wytrenuj zoptymalizowany model na zbiorze treningowym i ponownie ocen jego dokładność.

* Porównaj wyniki z oryginalnym modelem.
* Jakie wartości cech wpływają najbardziej na przewidywanie powagi wypadków (Severity)?
* Czy dokładność modelu wzrosła po optymalizacji parametrów? Jeśli tak, które parametry miały największy wpływ?
Jakie inne cechy można by dodać do modelu, aby poprawić jego jakość?


In [23]:
from pyspark.ml.feature import StringIndexer

idx = StringIndexer(inputCol="Severity",outputCol="Severity_tmp").fit(df)

df = idx.transform(df).drop("Severity").withColumnRenamed("Severity_tmp", "Severity")

df.show(2)

+--------------+-----------+--------------+---------------+-----------------+------------+--------+
|Temperature(F)|Humidity(%)|Visibility(mi)|Wind_Speed(mph)|Precipitation(in)|Distance(mi)|Severity|
+--------------+-----------+--------------+---------------+-----------------+------------+--------+
|          37.9|       97.0|           7.0|            3.5|             0.03|        0.01|     1.0|
|          37.4|      100.0|           3.0|            4.6|             0.02|        0.01|     1.0|
+--------------+-----------+--------------+---------------+-----------------+------------+--------+
only showing top 2 rows



In [28]:
assembler = VectorAssembler(inputCols=input_cols, outputCol="features")


In [40]:
model = DecisionTreeClassifier(featuresCol="features", labelCol=label, maxDepth=5)
pipeline = Pipeline(stages=[assembler, model])

In [41]:
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)
model = pipeline.fit(train_data)

In [42]:
predictions = model.transform(test_data)
predictions.select("features", label, "prediction").show()

+--------------------+--------+----------+
|            features|Severity|prediction|
+--------------------+--------+----------+
|[5.0,84.0,2.5,6.9...|     0.0|       0.0|
|[5.0,84.0,5.0,6.9...|     0.0|       0.0|
|[6.1,55.0,7.0,11....|     0.0|       0.0|
|[7.0,53.0,10.0,15...|     0.0|       0.0|
|[7.0,80.0,5.0,5.8...|     1.0|       0.0|
|[8.1,46.0,10.0,11...|     0.0|       0.0|
|[9.0,67.0,9.0,18....|     0.0|       0.0|
|[9.0,77.0,2.5,8.1...|     1.0|       0.0|
|[9.0,84.0,6.0,6.9...|     0.0|       0.0|
|[9.0,84.0,6.0,6.9...|     0.0|       0.0|
|[9.0,84.0,6.0,6.9...|     0.0|       0.0|
|[9.0,88.0,8.0,6.9...|     0.0|       0.0|
|[9.0,88.0,8.0,6.9...|     0.0|       0.0|
|[9.1,68.0,10.0,9....|     0.0|       0.0|
|[10.0,70.0,1.5,11...|     1.0|       0.0|
|[10.0,73.0,6.0,10...|     0.0|       0.0|
|[10.0,80.0,0.8,3....|     1.0|       0.0|
|[10.9,67.0,4.0,18...|     0.0|       0.0|
|[10.9,70.0,1.2,16...|     1.0|       0.0|
|[10.9,70.0,6.0,10...|     0.0|       0.0|
+----------

In [43]:
evaluator = MulticlassClassificationEvaluator(
   labelCol=label,
   predictionCol="prediction",
   metricName="accuracy"
)

accuracy = evaluator.evaluate(predictions)
print(f"Dokładność modelu: {accuracy:.2f}")


Dokładność modelu: 0.85


In [44]:
dt_model = model.stages[1]
print(dt_model.toDebugString)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_a98ec3623eaf, depth=0, numNodes=1, numClasses=4, numFeatures=6
  Predict: 0.0



+--------------------+-----+----------+--------------------+
|            features|label|prediction|         probability|
+--------------------+-----+----------+--------------------+
|[5.0,84.0,2.5,6.9...|  0.0|       0.0|[0.84567213718320...|
|[5.0,84.0,5.0,6.9...|  0.0|       0.0|[0.84567213718320...|
|[6.1,55.0,7.0,11....|  0.0|       0.0|[0.84567213718320...|
|[7.0,53.0,10.0,15...|  0.0|       0.0|[0.84567213718320...|
|[7.0,80.0,5.0,5.8...|  1.0|       0.0|[0.84567213718320...|
|[8.1,46.0,10.0,11...|  0.0|       0.0|[0.84567213718320...|
|[9.0,67.0,9.0,18....|  0.0|       0.0|[0.84567213718320...|
|[9.0,77.0,2.5,8.1...|  1.0|       0.0|[0.84567213718320...|
|[9.0,84.0,6.0,6.9...|  0.0|       0.0|[0.84567213718320...|
|[9.0,84.0,6.0,6.9...|  0.0|       0.0|[0.84567213718320...|
+--------------------+-----+----------+--------------------+
only showing top 10 rows

Dokładność modelu: 0.85
Struktura drzewa decyzyjnego:
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_19ca0

Dokładność zoptymalizowanego modelu: 0.85
Najlepsze parametry:
 - maxDepth: 5
 - maxBins: 32
Struktura najlepszego drzewa decyzyjnego:
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_69a9ab9bb848, depth=0, numNodes=1, numClasses=4, numFeatures=6
  Predict: 0.0

Wpływ cech na model:
 - Temperature(F): 0.0000
 - Humidity(%): 0.0000
 - Visibility(mi): 0.0000
 - Wind_Speed(mph): 0.0000
 - Precipitation(in): 0.0000
 - Distance(mi): 0.0000
